# Economic assumptions
### Table of Contents

* [1. Introduction ](#1.introduction)
* [2. Loading Economic assumption and computing annualized costs (marginal and fixed)](#2.Loading)
    * [2.2 General economic assumptions](#2.1.Loading)
    * [2.2 Production economic assumptions](#2.2.Loading)
    * [2.3 Compute variable and fixed annualized costs](#2.3.Loading)
* [3. Generation of assumption files for the Planning/operation tool](#3.Loading)
    * [3.1 Technical on the electric mix. ](#3.1.Loading)
    * [3.2 Generating file for operation ](#3.2.operation)
    * [3.3 Generating file for Planning ](#3.3.Planning)
* [4. A first evaluation of System Cost](#4.SystemCost)
    * [4.1 System cost computation](#4.1SystemCost)
* [5. Multi-zone case](#5.multizone)
    * [5.1 Production data](#5.1.multizone)
    * [5.2 Multizone Operation](#5.2.multizone)
    * [5.3 Multizone Planning](#5.3.multizone)


## 1. Introduction  <a class="anchor" id="1.introduction"></a>
In this file we learn how to compute a set of annualized costs that are used in the Planning tool
For each technology it is only two numbers :
 - variable cost [€/MWh]
 - fixed annualized [€/kW/year]

These are obtained from a set of general economic assumptions
 - discounting rate [%]
 - carbon tax [€/t]

and assumptions per production technology (eventually per area)
 - CAPEX [€/kW]
 - Cost of dismantling [€/kW]
 - fixed operation and maintenance cost [€/kW/year]
 - variable cost [€/MWh]
 - life length [year]
 - construction length [year]
 - CO2 Emissions [gCO2/kWh]


Before you start here you should be familiar with annualized cost of electricity (variable and fixed cost) and levelised cost of energy.
In [my post here on lcoe](https://www.energy-alternatives.eu/2020/08/20/decomposition-lcoe.html) you will find all sources for
formulae and economic assumptions (in french, if there are people that would be willing to contribute to a translation that would be great !!)
and also [this one on the role of power and enery in system cost](https://www.energy-alternatives.eu/2020/05/07/mix-de-production-delectricite-energie-et-puissance.html).
An old excel file [here](https://docs.google.com/spreadsheets/d/1wWWu0YwsvkzmEG8XDRnsIoEegmALqZL-9XPxWx3z2Qo/edit?usp=sharing) contains similar stuff.
Side remark : to make all this be interactive in jupyter notebook, I'm using package [qgrid](https://github.com/quantopian/qgrid).

In [ ]:
#region importation of modules
import os

InputFolder='Data/input/'
import pandas as pd
import qgrid # great package https://github.com/quantopian/qgrid
import numpy as np
#endregion

## 2. Loading Economic assumption and computing annualized costs (marginal and fixed) <a class="anchor" id="2.Loading"></a>

### 2.2 General economic assumptions <a class="anchor" id="2.1.Loading"></a>

In [ ]:
#region - general economic assumption
General = pd.read_csv(InputFolder+'GeneralEconomicAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
col_def = {}#  { 'A': { 'editable': True } }
qgrid_widget = qgrid.show_grid(General, show_toolbar=True, column_definitions=col_def)
qgrid_widget
#endregion

In [ ]:
General=qgrid_widget.get_changed_df()
General



### 2.2 Production economic assumptions  <a class="anchor" id="2.2.Loading"></a>

In [ ]:
#region - general production assumption
Production = pd.read_csv(InputFolder+'ProductionEconomicAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
Production=Production[Production["AREAS"]=="FR"]
qgrid_widget = qgrid.show_grid(Production, show_toolbar=True, column_definitions=col_def)
qgrid_widget

#endregion

In [ ]:
Production=qgrid_widget.get_changed_df()
Production

### 2.3 Compute variable and fixed annualized costs <a class="anchor" id="2.3.Loading"></a>
For each technology it is only two numbers :
 - variable cost [€/MWh]
 - fixed annualized [€/kW/year]

The formula used here is explained [in this post](https://www.energy-alternatives.eu/2020/08/20/decomposition-lcoe.html)

In [ ]:

r=(General.discountPercent[0]/100)
#r=0.05
#Production.CAPEX[6]=1100
#Production.CAPEX[5]=6000
Production=Production.assign(
    LLr = round((1+r)/r*(1-(1+r)**(-Production['LL'])),2))
Production = Production.assign(
    capacityCost = round(Production.CAPEX/Production.LLr+ Production.dismantling/(1+r)**(Production.LL) +Production.FOM,2)*1000,
    energyCost = round(Production.Variable + Production.CO2Emission*General.carbonTaxEurosPerT[0]/10**3,2)
)

Production
## Storage economic assumptions



## 3. Generation of assumption files for the Planning/operation tool <a class="anchor" id="3.Loading"></a>
### 3.1 Technical on the electric mix. <a class="anchor" id="3.1.Loading"></a>

In [ ]:
#region - general economic assumption
ProductionTech = pd.read_csv(InputFolder+'ProductionTechnicalAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
ProductionTech=ProductionTech[ProductionTech["AREAS"]=="FR"]
qgrid_widget = qgrid.show_grid(ProductionTech, show_toolbar=True, column_definitions=col_def)
qgrid_widget
ProductionTech=qgrid_widget.get_changed_df()
ProductionTechPlus=ProductionTech.merge(Production, how='inner', left_on=["AREAS","TECHNOLOGIES"], right_on=["AREAS","TECHNOLOGIES"])
ProductionTechPlus
#endregion

Assumptions on installed power and produced energy

### 3.2 Generating file for Operation <a class="anchor" id="3.2.operation"></a>

In [ ]:

SelectedCols=["TECHNOLOGIES","energyCost","capacity","EnergyNbhourCap"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Gestion-Simple_TECHNOLOGIES.csv',sep=',',decimal='.', index=False)

SelectedCols=["TECHNOLOGIES","energyCost","capacity","EnergyNbhourCap","RampConstraintPlus","RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Gestion-RAMP1Bis_TECHNOLOGIES.csv',sep=',',decimal='.', index=False)


### 3.3 Generating file for Planning <a class="anchor" id="3.3.Planning"></a>

In [ ]:
ProductionTechPlus=ProductionTechPlus.assign(
    minCapacity = ProductionTechPlus["capacity"]*0.9,
    maxCapacity = ProductionTechPlus["capacity"]*100
)

ProductionTechPlus.maxCapacity.loc[ProductionTechPlus.TECHNOLOGIES=="HydroRiver"]=ProductionTechPlus.capacity.loc[ProductionTechPlus.TECHNOLOGIES=="HydroRiver"]
ProductionTechPlus.maxCapacity.loc[ProductionTechPlus.TECHNOLOGIES=="HydroReservoir"]=ProductionTechPlus.capacity.loc[ProductionTechPlus.TECHNOLOGIES=="HydroReservoir"]

SelectedCols=["TECHNOLOGIES","energyCost","capacityCost","EnergyNbhourCap",	"minCapacity","maxCapacity"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Planning-Simple_TECHNOLOGIES.csv',sep=',',decimal='.', index=False)

SelectedCols=["TECHNOLOGIES","energyCost","capacityCost","EnergyNbhourCap","minCapacity","maxCapacity","RampConstraintPlus","RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Planning-RAMP1BIS_TECHNOLOGIES.csv',sep=',',decimal='.', index=False)

ProductionTechPlus.capacityCost[ProductionTechPlus.TECHNOLOGIES=="NewNuke"]/6000



## 4. A first evaluation of System Cost <a class="anchor" id="3.SystemCost"></a>
Here we propose a first evaluation of system cost, it is similar to
 -  that proposed [in this post on the role of power and enery in system cost](https://www.energy-alternatives.eu/2020/05/07/mix-de-production-delectricite-energie-et-puissance.html).
 - and the one in an old excel file [here](https://docs.google.com/spreadsheets/d/1wWWu0YwsvkzmEG8XDRnsIoEegmALqZL-9XPxWx3z2Qo/edit?usp=sharing) contains similar stuff.

For that purpose, we only need
 - to use results from the preceding section (alpha and beta)
 - make assumptions of the installed power and yearly produced energy for each production mean.

For now we are not looking at how the equilibrium is done between production and consumption.

### 4.1 System cost computation <a class="anchor" id="3.2SystemCost"></a>

## 5. Multi-zone case <a class="anchor" id="5.multizone"></a>

### 5.1 Production data <a class="anchor" id="5.1.multizone"></a>

In [ ]:
#region - general economic assumption
Production = pd.read_csv(InputFolder+'ProductionEconomicAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
qgrid_widget = qgrid.show_grid(Production, show_toolbar=True, column_definitions=col_def)
qgrid_widget

#endregion

In [ ]:
Production=qgrid_widget.get_changed_df()
Production

In [ ]:
r=(General.discountPercent[0]/100)
Production=Production.assign(
    LLr = round((1+r)/r*(1-(1+r)**(-Production['LL'])),2))
Production = Production.assign(
    capacityCost = round(Production.CAPEX/Production.LLr+ Production.dismantling/(1+r)**(Production.LL) +Production.FOM,2)*1000,
    energyCost = round(Production.Variable + Production.CO2Emission*General.carbonTaxEurosPerT[0]/10**3,2)
)

## Storage economic assumptions

In [ ]:
#region - general economic assumption
ProductionTech = pd.read_csv(InputFolder+'ProductionTechnicalAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
qgrid_widget = qgrid.show_grid(ProductionTech, show_toolbar=True, column_definitions=col_def)
qgrid_widget
ProductionTech=qgrid_widget.get_changed_df()
ProductionTechPlus=ProductionTech.merge(Production, how='inner', left_on=["AREAS","TECHNOLOGIES"], right_on=["AREAS","TECHNOLOGIES"])
ProductionTechPlus
#endregion


### 5.2 Multizone Operation <a class="anchor" id="5.2.multizone"></a>

In [ ]:
SelectedCols=["AREAS","TECHNOLOGIES","energyCost","capacity","EnergyNbhourCap","RampConstraintPlus","RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Gestion_MultiNode_DE-FR_AREAS_TECHNOLOGIES.csv',sep=',',decimal='.', index=False)

### 5.3 Multizone Planning <a class="anchor" id="5.3.multizone"></a>


In [ ]:
ProductionTechPlus=ProductionTechPlus.assign(
    minCapacity = ProductionTechPlus["capacity"]*0.9,
    maxCapacity = ProductionTechPlus["capacity"]*100
)



index=ProductionTechPlus["TECHNOLOGIES"]=="HydroReservoir"
ProductionTechPlus.maxCapacity.loc[index]=ProductionTechPlus.capacity[index]
index=ProductionTechPlus["TECHNOLOGIES"]=="HydroRiver"
ProductionTechPlus.maxCapacity.loc[index]=ProductionTechPlus.capacity[index]
SelectedCols=["AREAS","TECHNOLOGIES","energyCost","capacityCost","EnergyNbhourCap","minCapacity","maxCapacity","RampConstraintPlus","RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Planning_MultiNode_DE-FR_TECHNOLOGIES_AREAS.csv',sep=',',decimal='.', index=False)



In [76]:
#region importation of modules
import os
if os.path.basename(os.getcwd())=="BasicFunctionalities":
    os.chdir('..') ## to work at project root  like in any IDE
InputFolder='Data/input/'
import pandas as pd
import qgrid # great package https://github.com/quantopian/qgrid
import numpy as np
#endregion

## 2. Loading Economic assumption and computing annualized costs (marginal and fixed) <a class="anchor" id="2.Loading"></a>

### 2.2 General economic assumptions <a class="anchor" id="2.1.Loading"></a>

In [77]:
#region - general economic assumption
General = pd.read_csv(InputFolder+'GeneralEconomicAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
col_def = {}#  { 'A': { 'editable': True } }
qgrid_widget = qgrid.show_grid(General, show_toolbar=True, column_definitions=col_def)
qgrid_widget
#endregion

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [78]:
General=qgrid_widget.get_changed_df()
General



,discountPercent,carbonTaxEurosPerT
0,4,40


### 2.2 Production economic assumptions  <a class="anchor" id="2.2.Loading"></a>

In [79]:
#region - general production assumption
Production = pd.read_csv(InputFolder+'ProductionEconomicAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
Production=Production[Production["AREAS"]=="FR"]
qgrid_widget = qgrid.show_grid(Production, show_toolbar=True, column_definitions=col_def)
qgrid_widget

#endregion

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [80]:
Production=qgrid_widget.get_changed_df()
Production

,YEAR,AREAS,TECHNOLOGIES,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission
0,2019,FR,TAC,600,100,60,36,30,2,600
1,2019,FR,Coal,1100,100,40,36,30,2,1000
2,2019,FR,CCG,900,100,50,36,30,2,470
3,2019,FR,OldNuke,0,1000,7,150,60,0,0
4,2019,FR,NukeCarrene,1587,1000,7,150,60,0,0
5,2019,FR,NewNuke,5000,1000,7,150,80,10,0
6,2019,FR,WindOnShore,1300,50,0,45,30,2,0
7,2019,FR,WindOffShore,2500,100,0,114,30,3,0
8,2019,FR,HydroReservoir,1500,10,0,20,80,10,0
9,2019,FR,HydroRiver,1500,10,0,20,80,10,0


### 2.3 Compute variable and fixed annualized costs <a class="anchor" id="2.3.Loading"></a>
For each technology it is only two numbers :
 - variable cost [€/MWh]
 - fixed annualized [€/kW/year]

The formula used here is explained [in this post](https://www.energy-alternatives.eu/2020/08/20/decomposition-lcoe.html)

In [81]:

r=(General.discountPercent[0]/100)
#r=0.05
#Production.CAPEX[6]=1100
#Production.CAPEX[5]=6000
Production=Production.assign(
    LLr = round((1+r)/r*(1-(1+r)**(-Production['LL'])),2))
Production = Production.assign(
    capacityCost = round(Production.CAPEX/Production.LLr+ Production.dismantling/(1+r)**(Production.LL) +Production.FOM,2)*1000,
    energyCost = round(Production.Variable + Production.CO2Emission*General.carbonTaxEurosPerT[0]/10**3,2)
)

Production
## Storage economic assumptions



,YEAR,AREAS,TECHNOLOGIES,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission,LLr,capacityCost,energyCost
0,2019,FR,TAC,600,100,60,36,30,2,600,17.98,100200.0,84.0
1,2019,FR,Coal,1100,100,40,36,30,2,1000,17.98,128010.0,80.0
2,2019,FR,CCG,900,100,50,36,30,2,470,17.98,116890.0,68.8
3,2019,FR,OldNuke,0,1000,7,150,60,0,0,23.53,245060.0,7.0
4,2019,FR,NukeCarrene,1587,1000,7,150,60,0,0,23.53,312510.0,7.0
5,2019,FR,NewNuke,5000,1000,7,150,80,10,0,24.87,394430.0,7.0
6,2019,FR,WindOnShore,1300,50,0,45,30,2,0,17.98,132720.0,0.0
7,2019,FR,WindOffShore,2500,100,0,114,30,3,0,17.98,283880.0,0.0
8,2019,FR,HydroReservoir,1500,10,0,20,80,10,0,24.87,80750.0,0.0
9,2019,FR,HydroRiver,1500,10,0,20,80,10,0,24.87,80750.0,0.0


## 3. Generation of assumption files for the Planning/operation tool <a class="anchor" id="3.Loading"></a>
### 3.1 Technical on the electric mix. <a class="anchor" id="3.1.Loading"></a>

In [82]:
#region - general economic assumption
ProductionTech = pd.read_csv(InputFolder+'ProductionTechnicalAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
ProductionTech=ProductionTech[ProductionTech["AREAS"]=="FR"]
qgrid_widget = qgrid.show_grid(ProductionTech, show_toolbar=True, column_definitions=col_def)
qgrid_widget
ProductionTech=qgrid_widget.get_changed_df()
ProductionTechPlus=ProductionTech.merge(Production, how='inner', left_on=["AREAS","TECHNOLOGIES"], right_on=["AREAS","TECHNOLOGIES"])
ProductionTechPlus
#endregion

,YEAR_x,AREAS,TECHNOLOGIES,capacity,ProducedEnergy,EnergyNbhourCap,RampConstraintPlus,RampConstraintMoins,RampConstraintPlus2,RampConstraintMoins2,...,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission,LLr,capacityCost,energyCost
0,2019,FR,TAC,4000,3.2,0,0.00,0.00,0,0,...,600,100,60,36,30,2,600,17.98,100200.0,84.0
1,2019,FR,Coal,2000,1.0,0,0.06,0.06,0,0,...,1100,100,40,36,30,2,1000,17.98,128010.0,80.0
2,2019,FR,CCG,12000,38.6,0,0.06,0.06,0,0,...,900,100,50,36,30,2,470,17.98,116890.0,68.8
3,2019,FR,OldNuke,63000,413.0,0,0.04,0.04,0,0,...,0,1000,7,150,60,0,0,23.53,245060.0,7.0
4,2019,FR,NukeCarrene,0,0.0,0,0.00,0.00,0,0,...,1587,1000,7,150,60,0,0,23.53,312510.0,7.0
5,2019,FR,NewNuke,0,0.0,0,0.00,0.00,0,0,...,5000,1000,7,150,80,10,0,24.87,394430.0,7.0
6,2019,FR,WindOnShore,14900,29.0,0,0.00,0.00,0,0,...,1300,50,0,45,30,2,0,17.98,132720.0,0.0
7,2019,FR,WindOffShore,0,0.0,0,0.00,0.00,0,0,...,2500,100,0,114,30,3,0,17.98,283880.0,0.0
8,2019,FR,HydroReservoir,7000,33.0,2100,0.00,0.00,0,0,...,1500,10,0,20,80,10,0,24.87,80750.0,0.0
9,2019,FR,HydroRiver,10000,30.0,0,0.00,0.00,0,0,...,1500,10,0,20,80,10,0,24.87,80750.0,0.0


Assumptions on installed power and produced energy

### 3.2 Generating file for Operation <a class="anchor" id="3.2.operation"></a>

In [83]:

SelectedCols=["TECHNOLOGIES","energyCost","capacity","EnergyNbhourCap"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Gestion-Simple_TECHNOLOGIES.csv',sep=',',decimal='.', index=False)

SelectedCols=["TECHNOLOGIES","energyCost","capacity","EnergyNbhourCap","RampConstraintPlus","RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Gestion-RAMP1Bis_TECHNOLOGIES.csv',sep=',',decimal='.', index=False)


### 3.3 Generating file for Planning <a class="anchor" id="3.3.Planning"></a>

In [84]:
ProductionTechPlus=ProductionTechPlus.assign(
    minCapacity = ProductionTechPlus["capacity"]*0.9,
    maxCapacity = ProductionTechPlus["capacity"]*100
)

ProductionTechPlus.maxCapacity.loc[ProductionTechPlus.TECHNOLOGIES=="HydroRiver"]=ProductionTechPlus.capacity.loc[ProductionTechPlus.TECHNOLOGIES=="HydroRiver"]
ProductionTechPlus.maxCapacity.loc[ProductionTechPlus.TECHNOLOGIES=="HydroReservoir"]=ProductionTechPlus.capacity.loc[ProductionTechPlus.TECHNOLOGIES=="HydroReservoir"]

SelectedCols=["TECHNOLOGIES","energyCost","capacityCost","EnergyNbhourCap",	"minCapacity","maxCapacity"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Planning-Simple_TECHNOLOGIES.csv',sep=',',decimal='.', index=False)

SelectedCols=["TECHNOLOGIES","energyCost","capacityCost","EnergyNbhourCap","minCapacity","maxCapacity","RampConstraintPlus","RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Planning-RAMP1BIS_TECHNOLOGIES.csv',sep=',',decimal='.', index=False)

ProductionTechPlus.capacityCost[ProductionTechPlus.TECHNOLOGIES=="NewNuke"]/6000



/Users/robin.girard/opt/anaconda3/envs/energyalternatives/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


5    65.738333
Name: capacityCost, dtype: float64

## 4. A first evaluation of System Cost <a class="anchor" id="3.SystemCost"></a>
Here we propose a first evaluation of system cost, it is similar to
 -  that proposed [in this post on the role of power and enery in system cost](https://www.energy-alternatives.eu/2020/05/07/mix-de-production-delectricite-energie-et-puissance.html).
 - and the one in an old excel file [here](https://docs.google.com/spreadsheets/d/1wWWu0YwsvkzmEG8XDRnsIoEegmALqZL-9XPxWx3z2Qo/edit?usp=sharing) contains similar stuff.

For that purpose, we only need
 - to use results from the preceding section (alpha and beta)
 - make assumptions of the installed power and yearly produced energy for each production mean.

For now we are not looking at how the equilibrium is done between production and consumption.

### 4.1 System cost computation <a class="anchor" id="3.2SystemCost"></a>

## 5. Multi-zone case <a class="anchor" id="5.multizone"></a>

### 5.1 Production data <a class="anchor" id="5.1.multizone"></a>

In [85]:
#region - general economic assumption
Production = pd.read_csv(InputFolder+'ProductionEconomicAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
qgrid_widget = qgrid.show_grid(Production, show_toolbar=True, column_definitions=col_def)
qgrid_widget

#endregion

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [86]:
Production=qgrid_widget.get_changed_df()
Production

,YEAR,AREAS,TECHNOLOGIES,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission
0,2019,FR,TAC,600,100,60,36,30,2,600
1,2019,FR,Coal,1100,100,40,36,30,2,1000
2,2019,FR,CCG,900,100,50,36,30,2,470
3,2019,FR,OldNuke,0,1000,7,150,60,0,0
4,2019,FR,NukeCarrene,1587,1000,7,150,60,0,0
5,2019,FR,NewNuke,5000,1000,7,150,80,10,0
6,2019,FR,WindOnShore,1300,50,0,45,30,2,0
7,2019,FR,WindOffShore,2500,100,0,114,30,3,0
8,2019,FR,HydroReservoir,1500,10,0,20,80,10,0
9,2019,FR,HydroRiver,1500,10,0,20,80,10,0


In [87]:
r=(General.discountPercent[0]/100)
Production=Production.assign(
    LLr = round((1+r)/r*(1-(1+r)**(-Production['LL'])),2))
Production = Production.assign(
    capacityCost = round(Production.CAPEX/Production.LLr+ Production.dismantling/(1+r)**(Production.LL) +Production.FOM,2)*1000,
    energyCost = round(Production.Variable + Production.CO2Emission*General.carbonTaxEurosPerT[0]/10**3,2)
)

## Storage economic assumptions

In [88]:
#region - general economic assumption
ProductionTech = pd.read_csv(InputFolder+'ProductionTechnicalAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
qgrid_widget = qgrid.show_grid(ProductionTech, show_toolbar=True, column_definitions=col_def)
qgrid_widget
ProductionTech=qgrid_widget.get_changed_df()
ProductionTechPlus=ProductionTech.merge(Production, how='inner', left_on=["AREAS","TECHNOLOGIES"], right_on=["AREAS","TECHNOLOGIES"])
ProductionTechPlus
#endregion


,YEAR_x,AREAS,TECHNOLOGIES,capacity,ProducedEnergy,EnergyNbhourCap,RampConstraintPlus,RampConstraintMoins,RampConstraintPlus2,RampConstraintMoins2,...,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission,LLr,capacityCost,energyCost
0,2019,FR,TAC,4000,3.2,0,0.00,0.00,0,0,...,600,100,60,36,30,2,600,17.98,100200.0,84.0
1,2019,FR,Coal,2000,1.0,0,0.06,0.06,0,0,...,1100,100,40,36,30,2,1000,17.98,128010.0,80.0
2,2019,FR,CCG,12000,38.6,0,0.06,0.06,0,0,...,900,100,50,36,30,2,470,17.98,116890.0,68.8
3,2019,FR,OldNuke,63000,413.0,0,0.04,0.04,0,0,...,0,1000,7,150,60,0,0,23.53,245060.0,7.0
4,2019,FR,NukeCarrene,0,0.0,0,0.00,0.00,0,0,...,1587,1000,7,150,60,0,0,23.53,312510.0,7.0
5,2019,FR,NewNuke,0,0.0,0,0.00,0.00,0,0,...,5000,1000,7,150,80,10,0,24.87,394430.0,7.0
6,2019,FR,WindOnShore,14900,29.0,0,0.00,0.00,0,0,...,1300,50,0,45,30,2,0,17.98,132720.0,0.0
7,2019,FR,WindOffShore,0,0.0,0,0.00,0.00,0,0,...,2500,100,0,114,30,3,0,17.98,283880.0,0.0
8,2019,FR,HydroReservoir,7000,33.0,2100,0.00,0.00,0,0,...,1500,10,0,20,80,10,0,24.87,80750.0,0.0
9,2019,FR,HydroRiver,10000,30.0,0,0.00,0.00,0,0,...,1500,10,0,20,80,10,0,24.87,80750.0,0.0


### 5.2 Multizone Operation <a class="anchor" id="5.2.multizone"></a>

In [89]:
SelectedCols=["AREAS","TECHNOLOGIES","energyCost","capacity","EnergyNbhourCap","RampConstraintPlus","RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Gestion_MultiNode_DE-FR_AREAS_TECHNOLOGIES.csv',sep=',',decimal='.', index=False)

### 5.3 Multizone Planning <a class="anchor" id="5.3.multizone"></a>


In [90]:
ProductionTechPlus=ProductionTechPlus.assign(
    minCapacity = ProductionTechPlus["capacity"]*0.9,
    maxCapacity = ProductionTechPlus["capacity"]*100
)



index=ProductionTechPlus["TECHNOLOGIES"]=="HydroReservoir"
ProductionTechPlus.maxCapacity.loc[index]=ProductionTechPlus.capacity[index]
index=ProductionTechPlus["TECHNOLOGIES"]=="HydroRiver"
ProductionTechPlus.maxCapacity.loc[index]=ProductionTechPlus.capacity[index]
SelectedCols=["AREAS","TECHNOLOGIES","energyCost","capacityCost","EnergyNbhourCap","minCapacity","maxCapacity","RampConstraintPlus","RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Planning_MultiNode_DE-FR_TECHNOLOGIES_AREAS.csv',sep=',',decimal='.', index=False)



/Users/robin.girard/opt/anaconda3/envs/energyalternatives/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
